# High-level Algorithm Design with Qmod
# Workshop Part I - Language Concepts

In this workshop we will learn to use high-level quantum programming language concepts to design quantum algorithms. We will use the Qmod language to model the functionality, and the Classiq platform to synthesize it into gate-level descriptions, visualize the circuits, and execute them. We will focus on high-level quantum types and expressions in different evaluation modes.

Part I is a walk through Qmod's structure and constructs, as well as some of its unique high-level concepts. We will look closely at these constructs using small examples. In Part II we will combine some of these concepts in a complete quantum algorithm.

There are 5 code exercises in this notebook, split into two sections, *A* and *B*. In each exercise follow the instructions - complete the code snippet where indicated by a *TODO* comment, execute the code, and try to understand the results. Solutions are provided at the end of the notebook. Don't continue to the next exercise before you completed the previous one and compared your code and results against the solution.

## Section A *(15 minutes)*

### Warmup: A First Qmod Program

Let's start with a simple example to demonstrate the structure of Qmod code, as well as its synthesis and execution flow. We prepare and sample a **Bell state** - one of the most fundamental quantum states.

Further reading:
[Quantum Functions](https://docs.classiq.io/latest/qmod-reference/language-reference/functions/)

In [1]:
from classiq import *


@qfunc  # This decorator declares a quantum function
def create_bell_state(pair: QArray[QBit, 2]):
    H(pair[0])
    CX(pair[0], pair[1])


@qfunc  # Function 'main' is the entry point of our quantum program
def main(res: Output[QArray[QBit, 2]]):
    allocate(res)
    create_bell_state(res)

*<div class="alert alert-success">Quantum functions in Qmod are defined using a regular Python function, decorated with `qfunc`, and their parameters must be declared with type hints.</div>*

We can now compile with the SDK function `synthesize`. We get back an executable description called *quantum program* which we then execute on any simulation or quantum hardware. To manage the execution flow we use `ExecutionSession`. In our case, a simple sampling (using the default number of shots) of the quantum program will suffice.

In [2]:
qprog = synthesize(main)

show(qprog)  # Visualize the quantum program for analysis

# Execute and print the results:
with ExecutionSession(qprog) as es:
    res = es.sample()
    display(res.dataframe)

Quantum program link: https://platform.classiq.io/circuit/31e5FTDuDzoSqmCLHvRuAGquGAc


,res,count,probability,bitstring
0,"[0, 0]",1028,0.501953,00
1,"[1, 1]",1020,0.498047,11


*<div class="alert alert-primary">You should see in approximately 50% of the samples the bit vector 00 and in 50% the bit vector 11.</div>*

*<div class="alert alert-success">Synthesis is the process of compiling a high-level description to a gate-level description. The reduction is presented graphically. The executable format can be simulated using various simulation engines, or executed on quantum hardware of choice.</div>*

### Exercise 1: GHZ State

Based on the Bell state example above, prepare a **GHZ state** with 3 qubits. The GHZ state creates maximum entanglement between three qubits: (|000⟩ + |111⟩)/√2.

In [3]:
from classiq import *


@qfunc
def create_ghz_state(qubits: QArray[QBit, 3]):
    # TODO: Apply GHZ logic
    pass


@qfunc
def main(res: Output[QArray[QBit, 3]]):
    allocate(res)
    create_ghz_state(res)


# TODO: Synthesize the model, show, execute and print results
# Hint: Follow the same pattern as the demonstration above

*<div class="alert alert-primary">You should see in approximately 50% of the samples the bit vector 000 and in 50% the bit vector 111.</div>*

### Exercise 2: GHZ with Numeric Variables

Define a main function that calls the `create_ghz_state` (as implemented in Exercise 1) with a ***signed*** quantum number and outputs the results.

Synthesize it, analyze the quantum program, execute it and print the results. What do you expect the resulting value of `x` to be? 

Further reading: [Quantum Types](https://docs.classiq.io/latest/qmod-reference/language-reference/quantum-types/)

In [4]:
from classiq import *


@qfunc
def main(x: Output[QNum[3, SIGNED, 0]]):
    # TODO: Create GHZ state with QNum (gives 0 and -1)
    # Hint: Is it any different from the previous main implementation?
    pass


# TODO: Synthesize the model, show, execute and print results

*<div class="alert alert-secondary">You should see approximately 50% for 0 and 50% for -1.</div>*

*<div class="alert alert-info">In Qmod function arguments are automatically cast between `QNum` and `QArray[QBit]` (and also between other quantum types).
    The value of quantum variables is interpreted based on their type. The state $|111\rangle$ represents 7 for an unsigned integer, and -1 as signed integer (in two's complement encoding).</div>*

## Section B *(30 minutes)*

### Exercise 3: Arithmetic Expressions with Automatic Type Inference

#### Part A: Numeric type inference

Create and execute a quantum program that assigns a quantum arithmetic expression to a numeric variable:
1. Declare quantum numeric variables `a` and `b` as unsigned integers of size of 2.
2. Apply `hadamard_transform` on `a` and `b` (to put them in uniform superposition of all possible states).
3. Assign the value of `3*a + b` to `c`.
4. Synthesize, show, execute and print results.
 Inspect the printouts - what numeric attributes were inferred for variable `c`? Why?

Further reading: [Numeric Assignment](https://docs.classiq.io/latest/qmod-reference/language-reference/statements/numeric-assignment/)


In [5]:
from classiq import *


@qfunc
def main(a: Output[QNum[2]], b: Output[QNum[2]], c: Output[QNum]):
    allocate(a)
    allocate(b)
    # TODO: Put a and b in equal superposition

    # TODO:Assign the value of 3*a + b to c

    # Print out c's inferred size in qubits
    print(f"The size of c is {c.size}")


# TODO: Synthesize the model, show, execute and print results

*<div class="alert alert-secondary">The expression result ranges between 0 and 12. To represent all values variable `c` must be an unsigned integer of size 4 qubits or more.</div>*

*<div class="alert alert-info">In Qmod the size of numeric variables may be left unspecified. It is then automatically inferred to tightly fit all possible values of the expressions.</div>*

#### Part B: Numeric type inference with fixed-point fractions

Repeat *Part A*, only this time declaring `a` with 1 fraction digit and `b` with 2 fraction digits. How did the numeric attributes of `c` change? What are the corresponding sampled values for `c` in the result?


In [6]:
from classiq import *


@qfunc
def main(
    a: Output[QNum[2, UNSIGNED, 1]], b: Output[QNum[2, UNSIGNED, 2]], c: Output[QNum]
):
    allocate(a)
    allocate(b)
    # TODO: Put a and b in equal superposition

    # TODO: Assign the value of 3*a + b to c
    # Hint: Is it any different from the previous main implementation?

    # Print out the numeric attributes of the inferred type
    print("Numeric attributes of c:")
    print(
        f"size={c.size}, is_signed={c.is_signed}, fraction_digits={c.fraction_digits}"
    )


# TODO: Synthesize the model, show, execute and print results

*<div class="alert alert-secondary">Variable `c` should be of size 5, unsigend, and with 2 fraction digits. This is the minimal type that covers the expression's domain.</div>*

*<div class="alert alert-info">In Qmod, numeric variables can represent arbitrary fixed-point values. Arithemetic expression and type inference also accommodate for different decimal point locations.</div>*

### Exercise 4: Conditional Operations

Define a main function that initializes a quantum variable `x` (a 3-qubit signed number with 2 binary fraction digits) in an equal superposition of all states, then conditionally flips a single-qubit variable named `flag` when the value of x is less than 0.5. Inspect the execution results - how is `flag` entangled with `x`?

Further reading: [Control Statement](https://docs.classiq.io/latest/qmod-reference/language-reference/statements/control/)

In [7]:
from classiq import *


@qfunc
def main(x: Output[QNum[3, SIGNED, 2]], flag: Output[QBit]):
    allocate(x)
    hadamard_transform(x)

    allocate(flag)
    # TODO : Flip the state of flag if x < 0.5


# TODO: Synthesize the model, show, execute and print results

*<div class="alert alert-secondary">You should see that `x` is evenly distributed across the 8 values, and `flag` is flipped in 6 out of the 8 cases.</div>*

*<div class="alert alert-info">The `control` statement in Qmod is similar to classical *if* statement, where a statement block is applied conditionally, depending on a Boolean expression, and optionally an else-block is applied otherwise. The difference is that the operations are applied in superposition, corresponding to the condition.</div>*


### Exercise 5: Grover Algorithm using Quantum Struct and Constant Phase

The Grover search algorithm uses two kinds of conditional phase flips - one reflecting about the "good" states, and the other reflecting about the zero state. This is easy to express using fixed $\pi$ phase rotation under the required control condition. It is also convenient to encapsulate the problem variables in a quantum struct, so they can be passed around between functional units.

Create a quantum program that finds assignments for $a, b$ and $c \in \{0, 1, 2, 3\}$ that satisfy the equation $3a + b + 2c = 9$.
- Declare the varaibles as fiels of a quantum struct
- Define the phase oracle in terms of the problem condition
- Define the zero-reflection in the diffuser



In [8]:
from classiq import *
from classiq.qmod.symbolic import pi


class MyProblemVars(QStruct):
    # TODO: Declare the problem variables as quantum numeric fields
    dummy: (
        QBit  # Placeholder for the struct to be non-empty - replace with actual fields
    )


@qfunc
def phase_oracle(v: MyProblemVars):
    # TODO: Apply a phase flip if the state of v satisfies the equation (use field access in the form `v.a` etc.)
    pass


@qfunc
def zero_reflection(state: QNum):
    # TODO: Apply a phase flip if state is |0>
    pass


@qfunc
def grover_operator(v: MyProblemVars):
    phase_oracle(v)
    hadamard_transform(v)
    zero_reflection(v)
    hadamard_transform(v)


@qfunc
def main(v: Output[MyProblemVars]):
    allocate(v)
    hadamard_transform(v)
    for i in range(2):
        grover_operator(v)


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    display(res.dataframe)

Quantum program link: https://platform.classiq.io/circuit/31e5G2HEMuDsytsIQsoZyhDTSMJ


,v.dummy,count,probability,bitstring
0,0,1036,0.505859,0
1,1,1012,0.494141,1


*<div class="alert alert-secondary">There are 6 assignments to `a`, `b`, and `c` that satisfy the equation. They should be sampled with approximately equal probabilities.</div>*

*<div class="alert alert-info">In Qmod, a fixed phase (specifically $\pi$) can be introduced under `control` condition. This is a powerful idiom when combined with numeric quantum expressions.</div>*

### Exercise 6: Computing x² Using Phase Operations

#### Part A: Phase Encoding x²

Compute x² in the phase of its state. To actually view the phases of the states simulate the program using state-vector simulation.
Expand the quantum program visualization down to the gate-level implementation. How is the 'phase' statement synthesized?
Inspect the resulting phases of the different states. Do they match the phase expression?

Further reading: [Phase Statement](https://docs.classiq.io/latest/qmod-reference/language-reference/statements/phase/)

In [9]:
import numpy as np

from classiq import *
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences


@qfunc
def main(x: Output[QNum[3]]):
    allocate(x)
    # TODO: put x in uniform superposition

    # TODO: Encode x² into the phase


# Synthesize the model and show
qprog = synthesize(main)
show(qprog)

# Specify execution preferences for state vector simulation
preferences = ExecutionPreferences(
    num_shots=1,
    backend_preferences=ClassiqBackendPreferences(backend_name="simulator_statevector"),
)

# Execute and print results:
with ExecutionSession(qprog, preferences) as es:
    res = es.sample()
    res.dataframe.sort_values(by="x", inplace=True)  # Sort by x for better readability
    res.dataframe["phase"] = np.round(
        np.angle(res.dataframe.amplitude) / np.pi, 2
    )  # Add phase column in units of pi for convenience
    display(res.dataframe)

Quantum program link: https://platform.classiq.io/circuit/31e5GaZQaACMARem8VYHP5focxz


,x,amplitude,probability,bitstring,phase
0,0,1.0+0.0j,1.0,000,0.0



*<div class="alert alert-secondary">The resulting state phases should show x² rotation of the respective computational-state value, modulo 8 (determined by the domain of variable `x`). The steps are a 1/8 of a full $2\pi$ rotation</div>*

*<div class="alert alert-info">The `phase` statement can operate on an arbitrary polynomial over quantum numeric variables and compiles it into relative Z rotations of the respective states.</div>*

#### Bonus: Fourier Arithmetic

Create a quantum program that computes res = x² by encoding the result in the phase of a Fourier basis. Then transform the result back to the computational basis. Inspect the execution results to validate the correctness of your algorithm.

In [10]:
import numpy as np

from classiq import *


@qfunc
def main(x: Output[QNum[3]], res: Output[QNum[4]]):
    allocate(x)
    hadamard_transform(x)
    allocate(res)

    # TODO: Use within_apply with QFT and phase operations


# Synthesize the model and show
qprog = synthesize(main)
# show(qprog)

# Synthesize the model, show, execute and print results
with ExecutionSession(qprog) as es:
    res = es.sample()
    res.dataframe.sort_values(by="x", inplace=True)  # Sort by x for better readability
    display(res.dataframe)

,x,res,count,probability,bitstring
7,0,0,239,0.116699,0000000
4,1,0,256,0.125000,0000001
2,2,0,261,0.127441,0000010
6,3,0,244,0.119141,0000011
0,4,0,275,0.134277,0000100
1,5,0,267,0.130371,0000101
5,6,0,249,0.121582,0000110
3,7,0,257,0.125488,0000111


*<div class="alert alert-info">The `phase` statement can be used to implement modular arithmetic in the Fourier basis.</div>*

## Solutions

### Solution 1: GHZ State


In [11]:
from classiq import *


@qfunc
def create_ghz_state(qubits: QArray[QBit, 3]):
    # Apply GHZ logic
    H(qubits[0])
    CX(qubits[0], qubits[1])
    CX(qubits[1], qubits[2])


@qfunc
def main(res: Output[QArray[QBit, 3]]):
    allocate(res)
    create_ghz_state(res)


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.1 Results:")
    display(res.dataframe)

Quantum program link: https://platform.classiq.io/circuit/31e5Hm84N12unBX44OrbH14W57G
Ex.1 Results:


,res,count,probability,bitstring
0,"[1, 1, 1]",1036,0.505859,111
1,"[0, 0, 0]",1012,0.494141,000


### Solution 2: GHZ with Quantum Numbers


In [12]:
from classiq import *


@qfunc
def main(x: Output[QNum[3, SIGNED, 0]]):
    # Create GHZ state with QNum (gives 0 and -1)
    allocate(x)
    create_ghz_state(x)


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.2 Results:")
    display(res.dataframe)

Quantum program link: https://platform.classiq.io/circuit/31e5IGP6b3CyZMEO6umC7ncNL9j
Ex.2 Results:


,x,count,probability,bitstring
0,-1,1036,0.505859,111
1,0,1012,0.494141,000


### Solution 3: Arithmetic Expressions with Automatic Type Inference

#### Part A: Numeric type inference

In [13]:
from classiq import *


@qfunc
def main(a: Output[QNum[2]], b: Output[QNum[2]], c: Output[QNum]):
    allocate(a)
    allocate(b)
    # Put a and b in equal superposition
    hadamard_transform(a)
    hadamard_transform(b)

    # Assign the value of 3*a + b to c
    c |= 3 * a + b

    # Print out c's inferred size in qubits
    print(f"The size of c is {c.size}")


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.4A Results:")
    display(res.dataframe)

The size of c is 4
Quantum program link: https://platform.classiq.io/circuit/31e5IrvLtJVGMREqPYVOQpMUQvz
Ex.4A Results:


,a,b,c,count,probability,bitstring
0,3,3,12,155,0.075684,11001111
1,3,0,9,149,0.072754,10010011
2,2,3,9,138,0.067383,10011110
3,2,2,8,136,0.066406,10001010
4,2,0,6,135,0.065918,01100010
5,0,0,0,134,0.065430,00000000
6,3,1,10,131,0.063965,10100111
7,1,1,4,129,0.062988,01000101
8,1,2,5,129,0.062988,01011001
9,0,3,3,122,0.059570,00111100


#### Part B: Numeric type inference with fixed-point fractions

In [14]:
# Part B: Fractioned quantum numbers (explicit syntax)
from classiq import *


@qfunc
def main(
    a: Output[QNum[2, UNSIGNED, 1]], b: Output[QNum[2, UNSIGNED, 2]], c: Output[QNum]
):
    # Allocate a and b, then put in superposition
    allocate(a)
    allocate(b)
    hadamard_transform(a)
    hadamard_transform(b)

    # Assign the value of 3*a + b to c
    c |= 3 * a + b

    # Print out the numeric attributes of the inferred type
    print("Numeric attributes of c:")
    print(
        f"size: {c.size}, is_signed: {c.is_signed}, fraction_digits: {c.fraction_digits}"
    )


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.4B Results:")
    display(res.dataframe)

Numeric attributes of c:
size: 5, is_signed: False, fraction_digits: 2
Quantum program link: https://platform.classiq.io/circuit/31e5JJJTpXvIb2eYR1RWGKna5q5
Ex.4B Results:


,a,b,c,count,probability,bitstring
0,0.5,0.50,2.00,166,0.081055,010001001
1,1.5,0.75,5.25,144,0.070312,101011111
2,0.0,0.00,0.00,136,0.066406,000000000
3,0.0,0.25,0.25,135,0.065918,000010100
4,0.5,0.75,2.25,133,0.064941,010011101
5,1.0,0.25,3.25,129,0.062988,011010110
6,0.0,0.50,0.50,128,0.062500,000101000
7,0.0,0.75,0.75,128,0.062500,000111100
8,0.5,0.00,1.50,126,0.061523,001100001
9,1.5,0.25,4.75,122,0.059570,100110111


### Solution 4: Conditional Operations


In [15]:
from classiq import *


@qfunc
def main(x: Output[QNum[3, SIGNED, 2]], flag: Output[QBit]):
    allocate(x)
    hadamard_transform(x)

    allocate(flag)
    # Flip the state of flag if x < 0.5
    control(x < 0.5, lambda: X(flag))  # Flip the state of flag if x < 0.5


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.3 Results:")
    display(res.dataframe)

Quantum program link: https://platform.classiq.io/circuit/31e5K3eqzslULJigUVXpsXooQRj
Ex.3 Results:


,x,flag,count,probability,bitstring
0,-0.75,1,276,0.134766,1101
1,0.50,0,270,0.131836,0010
2,0.25,1,260,0.126953,1001
3,-0.50,1,255,0.124512,1110
4,0.75,0,251,0.122559,0011
5,0.00,1,247,0.120605,1000
6,-1.00,1,247,0.120605,1100
7,-0.25,1,242,0.118164,1111


### Solution 5: Grover Algorithm using Quantum Struct and Constant Phase

In [16]:
from classiq import *
from classiq.qmod.symbolic import pi


class MyProblemVars(QStruct):
    # Declare the problem variables as quantum numeric fields
    a: QNum[2]
    b: QNum[2]
    c: QNum[2]


@qfunc
def phase_oracle(v: MyProblemVars):
    # Apply a phase flip if the state of v satisfies the equation (use field access in the form `v.a` etc.)
    control(3 * v.a + v.b + 2 * v.c == 9, lambda: phase(pi))


@qfunc
def zero_reflection(state: QNum):
    # Apply a phase flip if state is |0>
    control(state == 0, lambda: phase(pi))


@qfunc
def grover_operator(v: MyProblemVars):
    phase_oracle(v)
    hadamard_transform(v)
    zero_reflection(v)
    hadamard_transform(v)


@qfunc
def main(v: Output[MyProblemVars]):
    allocate(v)
    hadamard_transform(v)
    for i in range(2):
        grover_operator(v)


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.5 Results:")
    display(res.dataframe)

Quantum program link: https://platform.classiq.io/circuit/31e5LqlXunOQkFo1LQyDSjcZ74G
Ex.5 Results:


,v.a,v.b,v.c,count,probability,bitstring
0,0,3,3,356,0.173828,111100
1,2,1,1,352,0.171875,010110
2,1,2,2,340,0.166016,101001
3,2,3,0,337,0.164551,001110
4,1,0,3,332,0.162109,110001
5,3,0,0,329,0.160645,000011
6,3,0,1,1,0.000488,010011
7,2,0,3,1,0.000488,110010


### Solution 6: Computing x² Using Phase Operations

#### Part A: Phase Encoding x²

In [17]:
import numpy as np

from classiq import *
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences


@qfunc
def main(x: Output[QNum[3]]):
    allocate(x)
    # Put x in uniform superposition
    hadamard_transform(x)

    # Encode x² into the phase
    phase(x**2, 2 * np.pi / 8)


# Synthesize the model and show
qprog = synthesize(main)
# show(qprog)

# Specify execution preferences for state vector simulation
preferences = ExecutionPreferences(
    num_shots=1,
    backend_preferences=ClassiqBackendPreferences(backend_name="simulator_statevector"),
)

# Execute and print results:
with ExecutionSession(qprog, preferences) as es:
    res = es.sample()
    print("Ex.6A Results:")
    res.dataframe.sort_values(by="x", inplace=True)  # Sort by x for better readability
    res.dataframe["phase"] = np.round(
        np.angle(res.dataframe.amplitude) / np.pi, 2
    )  # Add phase column in units of pi for convenience
    display(res.dataframe)

Ex.6A Results:


,x,amplitude,probability,bitstring,phase
1,0,0.353553+0.000000j,0.125,000,0.00
3,1,0.250000+0.250000j,0.125,001,0.25
6,2,-0.353553+0.000000j,0.125,010,1.00
5,3,0.250000+0.250000j,0.125,011,0.25
0,4,0.353553-0.000000j,0.125,100,-0.00
2,5,0.250000+0.250000j,0.125,101,0.25
7,6,-0.353553+0.000000j,0.125,110,1.00
4,7,0.250000+0.250000j,0.125,111,0.25


#### Bonus: Fourier Arithmetic


In [18]:
import numpy as np

from classiq import *


@qfunc
def main(x: Output[QNum[3]], res: Output[QNum[4]]):
    allocate(x)
    hadamard_transform(x)
    allocate(res)

    # Use within_apply with QFT and phase operations
    within_apply(
        lambda: qft(res),
        lambda: phase(
            res * (x**2), 2 * np.pi / (2**res.size)
        ),  # evaluates res += x**2  in the Fourier basis
    )


# Synthesize the model, show, execute and print results
qprog = synthesize(main)
# show(qprog)

with ExecutionSession(qprog) as es:
    res = es.sample()
    print("Ex.6B Results:")
    res.dataframe.sort_values(by="x", inplace=True)  # Sort by x for better readability
    display(res.dataframe)

Ex.6B Results:


,x,res,count,probability,bitstring
6,0,0,248,0.121094,0000000
2,1,1,261,0.127441,0001001
0,2,4,271,0.132324,0100010
5,3,9,251,0.122559,1001011
7,4,0,240,0.117188,0000100
4,5,9,256,0.125000,1001101
3,6,4,256,0.125000,0100110
1,7,1,265,0.129395,0001111
